In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os

from tqdm import tqdm

from GBP_Simulations.GBP.data import DataGenerator
from GBP_Simulations.GBP.gbp import calc_node_marginal, run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL
from GBP_Simulations.GBP.utilities import HiddenPrints
from GBP_Simulations.GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

### 1D Line

In [2]:
A, b = data_gen.get_1D_line_matrix_PSD(1000)



# file_path = 'GBP_Simulations/GBP/Raw_Datasets/data/input_M3500_g2o.g2o'
# data_gen.generate_SLAM_dataset_PSD(file_path=file_path)

# dataset = 'input_M3500_g2o'
# filepath_n = 'GBP_Simulations/GBP/Raw_Datasets/gbp_data'
# factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
# marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

# A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)


In [3]:
A

array([[ 1.01722832e+03, -1.08919712e+01, -0.00000000e+00, ...,
         0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
       [-1.08919712e+01,  1.02688341e+03, -2.51748352e-01, ...,
         0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
       [-0.00000000e+00, -2.51748352e-01,  1.06658334e+03, ...,
        -0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00, ...,
         1.03092367e+03, -1.26489199e+01, -0.00000000e+00],
       [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
        -1.26489199e+01,  1.03462444e+03,  5.29900384e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -0.00000000e+00,  5.29900384e+01,  1.00369622e+03]])

In [4]:
starting_mae = 10
b_prime = b
# b_prime = b * starting_mae / 1.843644159111673e-05

In [5]:
sync_convergence_threshold = 1*10**-0 # convergence threshold
convergence_type = 'all' #all
simulation_convergence_threshold = 1*10**-10

P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                      b_prime, 
                                                                                                      max_iter=1_000, 
                                                                                                      mae=True if convergence_type == 'mae' else False, 
                                                                                                      convergence_threshold=simulation_convergence_threshold,
                                                                                                      show=True)
final_mean_sync = list(mu_i)

print("================")
print(mu_i[0])

iteration: 1
0.027970184593395425
-----
iteration: 2
0.0003221729521995804
-----
iteration: 3
3.426295569819212e-05
-----
iteration: 4
3.7918841085657376e-06
-----
iteration: 5
4.445531516465942e-07
-----
iteration: 6
5.854938453104924e-08
-----
iteration: 7
4.725945910947499e-09
-----
iteration: 8
3.2927225399135965e-10
-----
iteration: 9
0.0
-----
0.0006571583746868953


In [6]:
print(A.shape)

(1000, 1000)


### Hardware

In [12]:
ASYNC_ITER = 1
NODE_UPDT_PE = 1
PEs = 1

async_convergence_threshold = 1*10**-5

In [13]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    _, _, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b_prime, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000,starting_mae=-1, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 0
1.847564721555494e-05
-----
iteration: 1
1.8475647215554942e-05
-----
iteration: 2
1.84381391492203e-05
-----
iteration: 3
1.8423605938651176e-05
-----
iteration: 4
1.8286289118533616e-05
-----
iteration: 5
1.8286289118533616e-05
-----
iteration: 6
1.8286289118533616e-05
-----
iteration: 7
1.828605634199976e-05
-----
iteration: 8
1.8273831460984406e-05
-----
iteration: 9
1.8189317632783704e-05
-----
iteration: 10
1.81540324222637e-05
-----
iteration: 11
1.8151531438733457e-05
-----
iteration: 12
1.8139932203673737e-05
-----
iteration: 13
1.8104890319735886e-05
-----
iteration: 14
1.7976327070156002e-05
-----
iteration: 15
1.7976327070156002e-05
-----
iteration: 16
1.7976194748494043e-05
-----
iteration: 17
1.795611043818356e-05
-----
iteration: 18
1.7955696041154855e-05
-----
iteration: 19
1.795504193830769e-05
-----
iteration: 20
1.795504193830769e-05
-----
iteration: 21
1.795504193830769e-05
-----
iteration: 22
1.795504193830769e-05
-----
iteration: 23
1.791871764951326e

In [15]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    _, _, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW(A, b_prime, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
0.0032169519534328526
-----
iteration: 2
0.0032169519534328526
-----
iteration: 3
0.0032169519534328526
-----
iteration: 4
0.0032169519534328526
-----
iteration: 5
0.00321609020743274
-----
iteration: 6
0.0032111630300200003
-----
iteration: 7
0.0032106794346751584
-----
iteration: 8
0.0032106794346751584
-----
iteration: 9
0.0032093818567228457
-----
iteration: 10
0.003204370522931379
-----
iteration: 11
0.0031975492715083744
-----
iteration: 12
0.003194768895240972
-----
iteration: 13
0.003194768895240972
-----
iteration: 14
0.0031902777267367463
-----
iteration: 15
0.0031848350302558495
-----
iteration: 16
0.0031794865326521845
-----
iteration: 17
0.0031733921508213607
-----
iteration: 18
0.003170930636592059
-----
iteration: 19
0.0031698958626955455
-----
iteration: 20
0.0031698958626955455
-----
iteration: 21
0.0031674485536764773
-----
iteration: 22
0.0031646116512971368
-----
iteration: 23
0.0031611923625412194
-----
iteration: 24
0.0031550050320356703
-----
iterati

In [14]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b_prime, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=1000, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
0.0032169519534328526
-----
iteration: 2
0.0032169519534328526
-----
iteration: 3
0.0032150999266801835
-----
iteration: 4
0.0032150999266801835
-----
iteration: 5
0.0032130401025978485
-----
iteration: 6
0.0032124294594526983
-----
iteration: 7
0.0032124294594526983
-----
iteration: 8
0.0032105088951706437
-----
iteration: 9
0.0032068707652407728
-----
iteration: 10
0.003200861966168198
-----
iteration: 11
0.003195431372665396
-----
iteration: 12
0.0031925368470389417
-----
iteration: 13
0.0031862822895056793
-----
iteration: 14
0.0031830422089714587
-----
iteration: 15
0.003178529746887871
-----
iteration: 16
0.003177844962441305
-----
iteration: 17
0.003175980026850584
-----
iteration: 18
0.003175980026850584
-----
iteration: 19
0.003172806579416829
-----
iteration: 20
0.0031650943128249697
-----
iteration: 21
0.0031624009288168486
-----
iteration: 22
0.0031624009288168486
-----
iteration: 23
0.003160915400399394
-----
iteration: 24
0.0031583408409076386
-----
iteration

### Test

In [11]:
from gbp

# initialize
P_ii, mu_ii, P_ij, mu_ij = initialize_m_ij(A, b)
num_nodes = A.shape[0]
N_i = find_neighbors_index(A)

# run the GaBP iterations
iteration = 0
early_end = False

# ========== Starting MAE ========== 
s = np.zeros(num_nodes)
m = np.zeros(num_nodes)
for i in range(num_nodes):
    s[i], m[i] = calc_node_marginal(i, P_ii, mu_ii, P_ij, mu_ij, N_i)

SyntaxError: invalid syntax (38046567.py, line 1)